In [2]:
import numpy as np
import parmed as pmd
import simtk.unit as su
import simtk.openmm as so
from pdbfixer import PDBFixer

# dependencies: numpy, parmed, openmm, pdbfixer, (nglview)

### Parameters

In [5]:
# parameters
tmp_filepath = "output.pdb"  # temporary pdb due to pdbfixer
# pdb_filepath = "../data/skempi/pdbs_mutated/1DAN_HL_UT_NU17A.pdb"  # input pdb filepath
pdb_filepath = '../data/pdbs/1A22.pdb'
pdb_filepath = '../data/pdbs_mutated/3QHY_A_B_WB13A.pdb'
subunits_chains = [['A'],
                   ['B']]  # list of chains (molecules) interacting (could be automatically parsed from filename)

# other random test (multiple mutations)
#pdb_filepath = "data/skempi/pdbs_mutated/2B2X_HL_A_VH50T_EH64K_QL28S_YL52D.pdb"
#subunits_chains = [['H','L'],['A']]

### Structure processing

In [6]:
# preprocess pdb file to fix numbering and formatting issues
pdb = pmd.load_file(pdb_filepath)
pdb.save(pdb_filepath, overwrite=True)

In [7]:
# repaire and prepare pdb structure
fixer = PDBFixer(filename=pdb_filepath)
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(False)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)

### Parametrization

In [8]:
# parameterize interactions
forcefield = so.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(fixer.topology, nonbondedMethod=so.app.NoCutoff)
param = pmd.openmm.load_topology(fixer.topology, system=system, xyz=fixer.positions)

ValueError: No template found for residue 259 (ALA).  The set of atoms matches NSER, but the bonds are different.

In [10]:
# debug display
param.to_dataframe()

,number,name,type,atomic_number,charge,mass,nb_idx,solvent_radius,screen,occupancy,...,rmin_14,epsilon_14,resname,resid,resnum,chain,segid,xx,xy,xz
0,-1,N,N1,7,0.1737,14.006720,0,0.0,0.0,0.0,...,1.82400,0.1700,PHE,0,0,A,,73.227000,31.442001,101.409998
1,-1,H,H1,1,0.1921,1.007947,0,0.0,0.0,0.0,...,0.60000,0.0157,PHE,0,0,A,,73.872333,31.744735,100.441828
2,-1,H2,H1,1,0.1921,1.007947,0,0.0,0.0,0.0,...,0.60000,0.0157,PHE,0,0,A,,72.820797,32.504072,101.788330
3,-1,H3,H1,1,0.1921,1.007947,0,0.0,0.0,0.0,...,0.60000,0.0157,PHE,0,0,A,,74.075112,31.169419,102.211246
4,-1,CA,C1,6,0.0733,12.010780,0,0.0,0.0,0.0,...,1.90800,0.1094,PHE,0,0,A,,72.119999,30.453000,101.239004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,-1,HB2,H5,1,0.0813,1.007947,0,0.0,0.0,0.0,...,1.38700,0.0157,SER,371,371,B,,11.375638,52.441964,110.352869
6053,-1,HB3,H5,1,0.0813,1.007947,0,0.0,0.0,0.0,...,1.38700,0.0157,SER,371,371,B,,12.825245,52.519302,109.018984
6054,-1,OG,O2,8,-0.6514,15.999430,0,0.0,0.0,0.0,...,1.72100,0.2104,SER,371,371,B,,13.422000,52.804999,110.892000
6055,-1,HG,H6,1,0.4474,1.007947,0,0.0,0.0,0.0,...,5.61231,0.0000,SER,371,371,B,,12.873325,53.544245,111.687336


In [11]:
# param.visualize()

### Pairwise non-bonded interactions potentials

In [12]:
# get indices of atoms for the 2 interacting subunits (could be more difficult if chain name doesn't match filename...)
ids0, ids1 = (np.where(param.to_dataframe()['chain'].isin(cids))[0] for cids in subunits_chains)

In [13]:
# sources
# https://en.wikipedia.org/wiki/Electrostatics
# https://en.wikipedia.org/wiki/Lennard-Jones_potential
# https://en.wikipedia.org/wiki/Combining_rules

# constants
eps0 = 8.8541878128e-12 * su.farad * su.meter**-1
e = 1.60217662e-19 * su.coulomb
N = 6.02214179e23 * su.mole**-1 # Avogadro

# scaling factors
k0 = (N * (e*e) / (4.0 * np.pi * eps0))

# get nonbonded interactions parameters for all atoms (Lennard-Jones and electrostatics)
epsilon = np.array([a.epsilon for a in param.atoms])
sigma = np.array([a.sigma for a in param.atoms])
charge = np.array([a.charge for a in param.atoms])

# pairwise epsilon with units
E = np.sqrt(epsilon[ids0].reshape(-1,1) * epsilon[ids1].reshape(1,-1)) * param.atoms[0].uepsilon.unit

# pairwise sigma with units
S = 0.5*(sigma[ids0].reshape(-1,1) + sigma[ids1].reshape(1,-1)) * param.atoms[0].usigma.unit

# pairwise partial charges
Q = charge[ids0].reshape(-1,1) * charge[ids1].reshape(1,-1)

# get atom coordinates and compute distance matrix between subunits
xyz = param.get_coordinates()[0]
D = np.linalg.norm(np.expand_dims(xyz[ids0], 1) - np.expand_dims(xyz[ids1], 0), axis=2) * su.angstrom

# compute nonbonded potential energies
U_LJ = (4.0 * E * (np.power(S/D, 12) - np.power(S/D, 6))).value_in_unit(su.kilojoule / su.mole)
U_el = (k0 * Q / D).value_in_unit(su.kilojoule / su.mole)

# debug print
print(f"U_LJ = {np.sum(U_LJ):.2f} kJ/mol; U_elec = {np.sum(U_el):.2f} kJ/mol")

U_LJ = -684.29 kJ/mol; U_elec = -490.91 kJ/mol


### Pairwise non-bonded interactions potentials on minimized structure

In [ ]:
ids0.shape

In [14]:
# create system
system = param.createSystem(nonbondedMethod=so.app.NoCutoff)

# setup MD engine
integrator = so.LangevinIntegrator(300*su.kelvin, 1/su.picosecond, 0.002*su.picoseconds)
platform = so.Platform.getPlatformByName('CPU')
simulation = so.app.Simulation(param.topology, system, integrator, platform)

# set atom coordinates
simulation.context.setPositions(param.get_coordinates()[0] * su.angstrom)

# minimize energy
simulation.minimizeEnergy()

# get state and new coordinates
state = simulation.context.getState(getPositions=True)
xyz_min = state.getPositions(asNumpy=True)

# compute distance matrix
D_min = np.linalg.norm(np.expand_dims(xyz_min._value[ids0], 1) - np.expand_dims(xyz_min._value[ids1], 0), axis=2) * xyz_min.unit

# compute nonbonded potential energies
U_LJ = (4.0 * E * (np.power(S/D_min, 12) - np.power(S/D_min, 6))).value_in_unit(su.kilojoule / su.mole)
U_el = (k0 * Q / D_min).value_in_unit(su.kilojoule / su.mole)

# debug print
print(f"U_LJ = {np.sum(U_LJ):.2f} kJ/mol; U_elec = {np.sum(U_el):.2f} kJ/mol")

U_LJ = -684.28 kJ/mol; U_elec = -1939.08 kJ/mol


In [25]:
xyz_min, xyz_min.shape

(Quantity(value=array([[ 7.29790314,  3.19615363, 10.24098658],
        [ 7.26747402,  3.29251963, 10.24247127],
        [ 7.38526573,  3.18792277, 10.19101289],
        ...,
        [ 1.29392818,  4.83367693, 11.12987776],
        [ 1.37402371,  4.79994714, 11.08474182],
        [ 1.11407383,  4.95640447, 10.76799476]]), unit=nanometer),
 (6057, 3))

In [63]:
ids0, ids1

(array([], dtype=int64), array([], dtype=int64))

In [38]:
xyz_min._value[ids0].shape, xyz_min._value[ids1].shape

((0, 3), (0, 3))

In [31]:
D_min

Quantity(value=array([], shape=(0, 0), dtype=float64), unit=nanometer)

In [11]:
# save pdb with non-minimized and minimized structures
#param.coordinates = np.stack([param.coordinates, xyz_min.value_in_unit(su.angstrom)], axis=0)
#param.save(tmp_filepath, overwrite=True)

In [12]:
#param.visualize()